# Notebook for reading in MRMS MESH data in grib format

### Import necessary libraries (xarray, cfgrib)

In [1]:
import xarray as xr
import cfgrib
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import dask.array as da
from dask import delayed
import dask.dataframe as dd
from dask.distributed import Client
client = Client()

2022-11-13 22:11:30,585 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-56m9_w_8', purging
2022-11-13 22:11:30,585 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-nukwpurg', purging
2022-11-13 22:11:30,585 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-oijsj7k_', purging
2022-11-13 22:11:30,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-xk1d90ub', purging
2022-11-13 22:11:30,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yy97njpf', purging
2022-11-13 22:11:30,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yxfmx4ya', purging
2022-11-13 22:11:30,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9hx4hr2l', purging

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 64,Total memory: 755.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43740,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 755.66 GiB
Comm: tcp://127.0.0.1:44880,Total threads: 8
Dashboard: http://127.0.0.1:36767/status,Memory: 94.46 GiB
Nanny: tcp://127.0.0.1:46249,


## Open all files with .grib2 extension stored in /home/scratch/mesh

In [ ]:
mesh_hist = xr.open_mfdataset('/home/scratch/mesh/*.grib2', engine = 'cfgrib', parallel=True) #Make sure to include engine = cfgrib since we are working with grib2 files
#and parallel = True to run using dask

Can't create file '/home/scratch/mesh/2015.grib2.923a8.idx'
Traceback (most recent call last):
  File "/anaconda3/envs/pyEAE/lib/python3.9/site-packages/cfgrib/messages.py", line 522, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/anaconda3/envs/pyEAE/lib/python3.9/contextlib.py", line 119, in __enter__
    return next(self.gen)
  File "/anaconda3/envs/pyEAE/lib/python3.9/site-packages/cfgrib/messages.py", line 488, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/home/scratch/mesh/2015.grib2.923a8.idx'
Can't read index file '/home/scratch/mesh/2015.grib2.923a8.idx'
Traceback (most recent call last):
  File "/anaconda3/envs/pyEAE/lib/python3.9/site-packages/cfgrib/messages.py", line 532, in from_indexpath_or_filestream
    index_mtime = os.path.getmtime(indexpath)
  File "/anaconda3/envs/pyEAE/lib/python3.9/genericpath.py", line 55, in getmtim

In [13]:
mesh_hist #Print out dataset

<xarray.Dataset>
Dimensions:         (time: 56794, latitude: 3500, longitude: 7000)
Coordinates:
  * time            (time) datetime64[ns] 2015-05-20T00:06:00 ... 2021-12-31T...
    step            timedelta64[ns] 00:00:00
    heightAboveSea  float64 500.0
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      (time) datetime64[ns] dask.array<chunksize=(5208,), meta=np.ndarray>
Data variables:
    unknown         (time, latitude, longitude) float32 dask.array<chunksize=(5208, 3500, 7000), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-11-13T21:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
mesh_daily_max = mesh_hist.resample(time = '24H', base = 12).max(dim = 'time') #Use xarray's resample method to resample to daily max values

In [15]:
mesh_daily_max #Test print dataset- should return 2555 days

<xarray.Dataset>
Dimensions:         (time: 2419, latitude: 3500, longitude: 7000)
Coordinates:
  * time            (time) datetime64[ns] 2015-05-19T12:00:00 ... 2021-12-31T...
    step            timedelta64[ns] 00:00:00
    heightAboveSea  float64 500.0
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
Data variables:
    unknown         (time, latitude, longitude) float32 dask.array<chunksize=(1, 3500, 7000), meta=np.ndarray>

In [24]:
mesh_daily_max.unknown.shape

(2419, 3500, 7000)

## Save MESH daily max dataset to netcdf

In [25]:
mesh_daily_max.to_netcdf('/home/scratch/jgoodin/mesh_daily_max.nc')

AttributeError: 'Dataset' object has no attribute 'to_grib'

2022-11-13 22:11:07,732 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,733 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,733 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,734 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,736 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,736 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,739 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,739 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,741 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,742 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,742 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-13 22:11:07,743 - distributed.nanny

### Assign lat/lon coords to variables

In [ ]:
lats = mesh_hist.latitude.values
lons = mesh_hist.longitude.values

### Assign mesh variable to plot (in this case, listed in dataset as 'unknown')

### Convert to units of inches (provided by dataset in mm)

In [15]:
mesh_plot = mesh_hist.unknown * 0.0393701 #can change variable name, currently unknown. Change to units of inches

In [14]:
mesh_plot.max()

<xarray.DataArray 'unknown' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    step            timedelta64[ns] 00:00:00
    heightAboveSea  float64 500.0

### Test plot day with large hail: Walter, AL 3/19/2018

In [18]:
#mesh_plot.sel(time = '2018-03-19T12:00:00.000000000').plot() #Select one time with large hail to test plot- Walter, AL 3/19/2018: 5.38" hail

## Plot on map of CONUS

In [ ]:
"""
fig = plt.figure(figsize= (12,8)) #Plot a figure
gs1 = gridspec.GridSpec(2, 1, width_ratios = [3], height_ratios=[1, .05], bottom=.05, top=.95, wspace=.1)
map_proj = ccrs.LambertConformal(central_longitude = -100, central_latitude = 35) #Set projection, central lat/lon
ax = plt.subplot(projection = map_proj)
#ax.set_title('Maximum diameter hail for 2005', fontsize = 22)
ax.add_feature(cfeature.LAND.with_scale('10m')) #Add land
countries_shp = shpreader.natural_earth(resolution='50m',
                                     category='cultural',
                                     name='admin_0_countries')
for country, info in zip(shpreader.Reader(countries_shp).geometries(), 
                             shpreader.Reader(countries_shp).records()):
        if info.attributes['NAME_LONG'] != 'United States':

            ax.add_geometries([country], ccrs.PlateCarree(),
                             facecolor='lightgrey', edgecolor='k', zorder=6)
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor='k', 
                                                facecolor='None'), zorder=8) 
ax.add_feature(cfeature.BORDERS.with_scale('10m')) #Add state borders
ax.add_feature(cfeature.STATES.with_scale('10m'), facecolor = 'none') #Add US states
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '50m', edgecolor='face', 
                                                facecolor='lightsteelblue'), zorder=6)
lakes_shp = shpreader.natural_earth(resolution='50m',
                                     category='physical',
                                     name='lakes')
for lake, info in zip(shpreader.Reader(lakes_shp).geometries(), 
                             shpreader.Reader(lakes_shp).records()):
        name = info.attributes['name']
        if name == 'Lake Superior' or name == 'Lake Michigan' or \
           name == 'Lake Huron' or name == 'Lake Erie' or name == 'Lake Ontario':
            
            ax.add_geometries([lake], ccrs.PlateCarree(),
                             facecolor='lightsteelblue', edgecolor='k', zorder=6)
ax.set_extent([240,287,22,50]) #Set lat/lon boundaries
lrFig = ax.pcolormesh(lons, lats, mesh_test, cmap = 'YlOrRd', transform = ccrs.PlateCarree())
#ax.text(-126, 31, 'SON', fontsize = 62, va='bottom', ha='center', rotation='vertical', transform=ccrs.PlateCarree())
#gls = ax.gridlines(draw_labels=True, dms = True, x_inline=False, y_inline=False, color="black", linestyle="dotted", zorder = 10)
#gls.top_labels=False
#gls.right_labels=False
#gls.xlabel_style = {'size': 18, 'rotation': 0}
#gls.xpadding = 12.0
#gls.ylabel_style = {'size': 18}
#plt.title("HAIL ≥ 1.57 in (END 8.5)", fontsize = 55)
ax1 = plt.subplot(gs1[1, 0])
cb = plt.colorbar(lrFig, cax = ax1, orientation = 'horizontal', extend = 'max')
#cb.set_label('Days', fontsize = 30)
cb.ax.tick_params(labelsize = 30)
plt.subplots_adjust(bottom=0.1)
#plt.savefig('/home/scratch/jgoodin/seasonal_mesh_climos/max/end8p5_4cm_SON_max.png')
"""